This lab on Linear Regression is a python adaptation of p. 110-120 of "Introduction to Statistical Learning with Applications in R" by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani. 
It is based on the work of R. Jordan Crouser at Smith College for SDS293: Machine Learning (Spring 2016).


# 3.6.1 Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# Tells matplotlib to display images inline instead of a new window
%matplotlib inline
import statsmodels.api as sm

We'll start by importing the data from [Boston.csv](http://www.science.smith.edu/~jcrouser/SDS293/data/Boston.csv) into a pandas dataframe:

In [ ]:
from_smith_edu = False # or True 

if not from_smith_edu:
    from sklearn.datasets import load_boston
    ds = load_boston()
    colnames = [line.split()[1].lower() for line in ds['DESCR'].split('\n') if "    -" in line]
    vars = np.concatenate([ds.data, ds.target[:,None]], axis=-1)
    df = pd.DataFrame(vars, columns=colnames)
    df.__doc__=ds['DESCR']
else:
    ## alternatively we could use this:
    url = 'https://www.science.smith.edu/~jcrouser/SDS293/data/Boston.csv'
    df = pd.read_csv(url, index_col=0)

print(df.head())
print(df.describe())

In [ ]:
print(df.__doc__)

# 3.6.2 Simple Linear Regression

Now let's fit a simple linear model (`OLS` - for "ordinary least squares" method) with `medv` as the response and `lstat` as the predictor:

In [ ]:
lm = sm.OLS.from_formula('medv ~ lstat', df)
result = lm.fit()

To get detailed information about the model, we can print the results of a call to the `.summary()` method:

In [ ]:
print(result.summary().as_text())

Want individual attributes? You can access them independently like this:

In [ ]:
result.rsquared, result.fvalue, result.params.Intercept, result.params.lstat

For a complete list of attributes and methods of a `RegressionResults` object, see: http://statsmodels.sourceforge.net/devel/generated/statsmodels.regression.linear_model.RegressionResults.html?highlight=regressionresults

For quick help use:

In [ ]:
result?

Now let's try making some predictions using this model. First, we'll set up a dataframe containing the `lstat` values for which we want to predict a response:

In [ ]:
new = pd.DataFrame([[1, 5], [1, 10], [1, 15]], columns=['Intercept', 'lstat'])

Now we just call the `.predict()` method:

In [ ]:
result.predict(new)

Technically those are the right prediction values, but maybe it would be good to have the confidence intervals along with them. Let's write a little helper function to get that and package it all up:

In [ ]:
Sigma=result.cov_params()
(new.dot(Sigma)*new).sum(axis=1)

In [ ]:
result.cov_params()

The covariance of the `intercept` term and the `lstat` coefficient is negative. Student-t distribution is in the `scipy.stats` module.

In [ ]:
from  scipy.stats import t as student_t
def predict(res, new, alpha=0.05):
    fit = pd.DataFrame(res.predict(new), columns=['fit'])
    # the variance of the predicted value is <new Sigma, new> (new is a row vector).
    sigma = (new.dot(res.cov_params())*new).sum(axis=1)**0.5
    # isf is the inverse survival function, 
    # that is the inverse of 1-F, where F is the distribution function
    t = student_t(df=res.df_resid, scale=sigma).isf(alpha/2)
    fit['lower'] = fit.fit-t
    fit['upper'] = fit.fit+t
    return fit
    

This is a confidence in the expected value of the response (the value of the regression function). Since new observations at these levels of `lstat` also contains the noise term, the confidence intervals for the new observed response values are much higher. 

In [ ]:
def predict(res, new, interval='confidence', alpha=0.05):
    fit = pd.DataFrame(res.predict(new), columns=['fit'])
    sigma2 = (new.dot(res.cov_params())*new).sum(axis=1)
    if interval=='prediction':
        sigma2 += res.scale
    sigma = sigma2**.5
    t = student_t(df=res.df_resid, scale=sigma).isf(alpha/2)
    fit['lower'] = fit.fit-t
    fit['upper'] = fit.fit+t
    return fit

In [ ]:
print(predict(result, new)) 
print(predict(result, new, interval='prediction')) 

Seaborn is a Python visualization library based on matplotlib that provides a high-level interface for drawing attractive statistical graphics.

In [ ]:
import seaborn as sns

We will now plot `medv` and `lstat` along with the least squares regression line using the `regplot()` function. We can define the color of the fit line using `line_kws` ("line keywords"):

In [ ]:
sns.regplot(x='lstat', y='medv', data=df, line_kws = {"color": "r"}, ci=None)
plt.grid()
plt.show()

We can also plot the residuals against the fitted values:

In [ ]:
fitted_values = pd.Series(result.fittedvalues, name="Fitted Values")
residuals = pd.Series(result.resid, name="Residuals")
sns.regplot(x=fitted_values, y=residuals, fit_reg=False)
plt.grid()
plt.show()

Perhaps we want studentized residuals instead? (This is almost the same as the residuals normalized to have unit norm, the latter is available as `.resid_pearson`)

In [ ]:
sns.scatterplot(x=np.arange(result.nobs), 
                y=result.get_influence().resid_studentized-result.resid_pearson)
plt.grid()
plt.show()

In [ ]:
s_residuals = pd.Series(result.get_influence().resid_studentized , name="S. Residuals")
sns.regplot(x=fitted_values, y=s_residuals,  fit_reg=False)
plt.grid()
plt.show()

We can also look for points with high leverage:

In [ ]:
from statsmodels.stats.outliers_influence import OLSInfluence
leverage = pd.Series(OLSInfluence(result).hat_matrix_diag, name = "Leverage")
sns.regplot(x=leverage, y=s_residuals,  fit_reg=False)
plt.grid()
plt.show()

#  3.6.3 Multiple Linear Regression

In order to fit a multiple linear regression model using least squares, we again use the `from_formula()` function. The syntax `from_formula(y∼x1+x2+x3)` is used to fit a model with three predictors, `x1`, `x2`, and `x3`. The `summary()` function now outputs the regression coefficients for all the predictors.

In [ ]:
model = sm.OLS.from_formula('medv ~ lstat + age', df)
result = model.fit()
print(result.summary())

The Boston data set contains 13 variables, and so it would be cumbersome to have to type all of these in order to perform a regression using all of the predictors. Instead, we can use the following short-hand:

In [ ]:
# All columns (except medv, which is our response)
model = sm.OLS.from_formula('medv ~ ' + '+'.join(df.columns.difference(['medv'])), df)
result = model.fit()
print(result.summary())

Note that we used the syntax `.join(df.columns.difference(['medv']))` to exclude the response variable above. We can use the same napproach to perform a regression using just a subset of the predictors? For example, in the above regression output, `age` and `indus` have a high p-value. So we may wish to run a regression excluding these predictors:

In [ ]:
# All columns (except medv age and indus)
model = sm.OLS.from_formula('medv ~ ' + '+'.join(df.columns.difference(['medv', 'age', 'indus'])), df)
result = model.fit()
print(result.summary())

# 3.6.4 Interaction Terms

It is easy to include interaction terms in a linear model using the `.from_formula()` function. The syntax `lstat:black` tells Python to include an interaction term between `lstat` and `black`. The syntax `lstat*age` simultaneously includes `lstat`, `age`, and the interaction term `lstat*age` as predictors; it is a shorthand for `lstat+age+lstat:age`.

In [ ]:
print(sm.OLS.from_formula('medv ~ lstat*age', df).fit().summary())

# 3.6.5 Non-linear Transformations of the Predictors

The `.from_formula()` function can also accommodate non-linear transformations of the predictors. For instance, given a predictor `X`, we can create a predictor `X^2` using `np.square(X)`. We now perform a regression of `medv` onto `lstat` and `lstat^2`.

In [ ]:
lm.fit2 = sm.OLS.from_formula('medv ~ lstat + np.square(lstat)', df).fit()
print(lm.fit2.summary())

The near-zero p-value associated with the quadratic term suggests that it leads to an improved model. We use the `anova_lm()` function to further quantify the extent to which the quadratic fit is superior to the linear fit.

In [ ]:
lm.fit = sm.OLS.from_formula('medv ~ lstat', df).fit()
print(sm.stats.anova_lm(lm.fit, lm.fit2))

Here Model 0 represents the linear submodel containing only one predictor, `lstat`, while Model 1 corresponds to the larger quadraticmodel that has two predictors, `lstat` and `lstat2`. The `anova_lm()` function performs a hypothesis test comparing the two models. The null hypothesis is that the two models fit the data equally well, and the alternative hypothesis is that the full model is superior. 

The F-statistic is 135 and the associated p-value is virtually zero. This provides very clear evidence that the model containing the predictors `lstat` and `lstat2` is far superior to the model that only contains the predictor `lstat`. This is not surprising, since earlier we saw evidence for non-linearity in the relationship between `medv` and `lstat`. 

If we type:

In [ ]:
fitted_values = pd.Series(lm.fit2.fittedvalues, name="Fitted Values")
residuals = pd.Series(lm.fit2.get_influence().resid_studentized, name="S. Residuals")
sns.regplot(x=fitted_values, y=s_residuals,  fit_reg=False)
plt.grid()
plt.show()

then we see that when the `lstat2` term is included in the model, there is little discernible pattern in the residuals.

In order to create a cubic fit, we can include a predictor of the form `np.power(x, 3))`. However, this approach can start to get cumbersome for higher order polynomials. A better approach involves using list comprehension inside a `.join()`. For example, the following command produces a fifth-order polynomial fit:

In [ ]:
formula_5 = 'medv ~ ' + '+'.join(['np.power(lstat,' + str(i) + ')' for i in range(1,6)])
print(formula_5)
print(sm.OLS.from_formula(formula_5, df).fit().summary())

Of course, we are in no way restricted to using polynomial transformations of the predictors. Here we try a log transformation.

In [ ]:
print(sm.OLS.from_formula('medv ~ np.log(rm)', df).fit().summary())

# 3.6.6 Qualitative Predictors

We will now examine the [`Carseats`](http://www.science.smith.edu/~jcrouser/SDS293/data/Carseats.csv) data that we talked about earlier in class. We will attempt to predict `Sales` (child car seat sales) in 400 locations based on a number of predictors.

In [ ]:
if from_smith_edu:
    df2 = pd.read_csv('Carseats.csv')
else:
    import statsmodels.datasets as smd
    df2 = smd.get_rdataset('Carseats', 'ISLR')
    df2.data.__doc__= df2.__doc__
    df2 = df2.data
df2.head()

In [ ]:
print(df2.__doc__)

The `Carseats` data includes qualitative predictors such as `Shelveloc`, an indicator of the quality of the shelving location—that is, the space within a store in which the car seat is displayed—at each location. The predictor `Shelveloc` takes on three possible values, `Bad`, `Medium`, and `Good`.

Given a qualitative variable such as `Shelveloc`, Python generates dummy variables automatically. Below we fit a multiple regression model that includes some interaction terms.

In [ ]:
formula_carseat = 'Sales ~ Income:Advertising+Price:Age + ' + " + ".join(df2.columns.difference(['Sales']))
print(formula_carseat)
print(sm.OLS.from_formula(formula_carseat, df2).fit().summary())

To learn how to set other coding schemes (or _contrasts_), see: http://statsmodels.sourceforge.net/devel/examples/notebooks/generated/contrasts.html